In [ ]:
import pyspark.ml.feature
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline 

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
df = spark.read.csv("politifact_news.csv",header=True, sep = ',',multiLine=True, escape = "\"",inferSchema =True)

In [ ]:
#df = df.select('news_id')

In [ ]:
df.dtypes

[('news_id', 'int'),
 ('url', 'string'),
 ('text', 'string'),
 ('num_images', 'int'),
 ('domain', 'string'),
 ('publish_date', 'string'),
 ('fake', 'int'),
 ('authors', 'string')]

In [ ]:
df_v = df.select('text','num_images','domain','publish_date','fake')

In [ ]:
df_v.toPandas()['text'].isnull().sum()

153

In [ ]:
df_v = df_v.dropna(subset=('text','domain'))

In [ ]:
tokenizer_text = Tokenizer(inputCol='text',outputCol='text1')
stopwords_remover_text = StopWordsRemover(inputCol='text1',outputCol='text2')
vectorizer_text = CountVectorizer(inputCol='text2',outputCol='text3')
idf_text = IDF(inputCol='text3',outputCol='text_vc')

In [ ]:
tokenizer_domain = Tokenizer(inputCol='domain',outputCol='domain1')
stopwords_remover_domain = StopWordsRemover(inputCol='domain1',outputCol='domain2')
vectorizer_domain = CountVectorizer(inputCol='domain2',outputCol='domain3')
idf_domain = IDF(inputCol='domain3',outputCol='domain_vc')

In [ ]:
(trainDF,testDF) = df_v.randomSplit((0.7,0.3),seed=42)

In [ ]:
assembler = VectorAssembler(inputCols=['text_vc','num_images'],outputCol="features")

In [ ]:
lr = LogisticRegression(featuresCol='features',labelCol='fake')

In [ ]:
pipeline = Pipeline(stages=[tokenizer_text,stopwords_remover_text,vectorizer_text,idf_text,
                            tokenizer_domain,stopwords_remover_domain,vectorizer_domain,idf_domain,
                            assembler,lr])

In [ ]:
lr_model = pipeline.fit(trainDF)

21/10/21 17:15:10 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
21/10/21 17:15:11 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
21/10/21 17:15:12 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/10/21 17:15:12 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/10/21 17:15:12 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
21/10/21 17:15:12 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
21/10/21 17:15:12 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
21/10/21 17:15:12 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
21/10/21 17:15:12 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
21/10/21 17:15:13 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
21/10/21 17:15:13 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
21/10/21 17:15:

In [ ]:
predictions = lr_model.transform(testDF)

In [ ]:
predictions = predictions.select('fake','prediction')

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol='fake',predictionCol='prediction',metricName='accuracy')

In [ ]:
accuracy = evaluator.evaluate(predictions)

21/10/21 17:15:18 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


In [ ]:
accuracy

0.8251748251748252